### 🧠 What is Query Decomposition?
Query decomposition is the process of taking a complex, multi-part question and breaking it into simpler, atomic sub-questions that can each be retrieved and answered individually.

#### ✅ Why Use Query Decomposition?

- Complex queries often involve multiple concepts

- LLMs or retrievers may miss parts of the original question

- It enables multi-hop reasoning (answering in steps)

- Allows parallelism (especially in multi-agent frameworks)

In [7]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

from langchain_openai import ChatOpenAI  # or ChatGroq
from langchain_core.prompts import PromptTemplate

from langchain_classic.chains import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableMap


In [8]:
# Step 1: Load and embed the document
loader = TextLoader("langchain_crewai_dataset.txt")
docs = loader.load()

splitter = RecursiveCharacterTextSplitter(chunk_size=300, chunk_overlap=50)
chunks = splitter.split_documents(docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(chunks, embedding)
retriever = vectorstore.as_retriever(search_type="mmr", search_kwargs={"k": 4, "lambda_mult": 0.7})

In [12]:
import os
from langchain.chat_models import init_chat_model
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=init_chat_model(model="groq:llama-3.1-8b-instant")
llm

ChatGroq(profile={'max_input_tokens': 131072, 'max_output_tokens': 8192, 'image_inputs': False, 'audio_inputs': False, 'video_inputs': False, 'image_outputs': False, 'audio_outputs': False, 'video_outputs': False, 'reasoning_output': False, 'tool_calling': True}, client=<groq.resources.chat.completions.Completions object at 0x000001DE74A8B4D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DE74CC4190>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [13]:
# Step 3: Query decomposition
decomposition_prompt = PromptTemplate.from_template("""
You are an AI assistant. Decompose the following complex question into 2 to 4 smaller sub-questions for better document retrieval.

Question: "{question}"

Sub-questions:
""")
decomposition_chain = decomposition_prompt | llm | StrOutputParser()

In [14]:
query = "How does LangChain use memory and agents compared to CrewAI?"
decomposition_question=decomposition_chain.invoke({"question": query})


In [15]:
print(decomposition_question)

To facilitate better document retrieval, I can decompose the complex question into the following sub-questions:

1. **What is LangChain's architecture and how does it utilize memory in conversational AI?**

   This sub-question aims to understand the memory component of LangChain and how it's designed to handle conversational tasks.

2. **How do agents interact with the memory component in LangChain?**

   This sub-question delves into the specifics of how agents in LangChain engage with the memory, highlighting their role in the system.

3. **What is CrewAI's architecture and how does it utilize memory and agents in conversational AI?**

   This sub-question provides context on CrewAI by understanding its overall architecture and how it employs memory and agents in conversational AI tasks.

4. **How does LangChain's use of memory and agents compare to CrewAI in terms of performance, scalability, or other relevant metrics?**

   This sub-question aims to identify key differences or sim

In [16]:
# Step 4: QA chain per sub-question
qa_prompt = PromptTemplate.from_template("""
Use the context below to answer the question.

Context:
{context}

Question: {input}
""")
qa_chain = create_stuff_documents_chain(llm=llm, prompt=qa_prompt)

In [17]:
# Step 5: Full RAG pipeline logic
def full_query_decomposition_rag_pipeline(user_query):
    # Decompose the query
    sub_qs_text = decomposition_chain.invoke({"question": user_query})
    sub_questions = [q.strip("-•1234567890. ").strip() for q in sub_qs_text.split("\n") if q.strip()]
    
    results = []
    for subq in sub_questions:
        docs = retriever.invoke(subq)
        result = qa_chain.invoke({"input": subq, "context": docs})
        results.append(f"Q: {subq}\nA: {result}")
    
    return "\n\n".join(results)

In [18]:
# Step 6: Run
query = "How does LangChain use memory and agents compared to CrewAI?"
final_answer = full_query_decomposition_rag_pipeline(query)
print("✅ Final Answer:\n")
print(final_answer)

✅ Final Answer:

Q: Here are 4 sub-questions that can help decompose the complex question:
A: It seems like you've provided a context related to LangChain and CrewAI, but you haven't asked a question yet. However, I can help you break down the complex topic into the 4 sub-questions you mentioned. Here are the sub-questions:

1. Can you explain the composition and reusability of chains in LangChain using standard patterns like Stuff, Map-Reduce, and Refine?
2. What types of workflows benefit from specialization and collaboration in CrewAI?
3. What are some common applications of LangChain agents, such as web search, calculators, and code execution environments?
4. How does the planner-executor model in LangChain agents enable dynamic decision-making, branching logic, and context-aware memory use across steps?

Q: **What is LangChain, and what are its key components (memory and agents)?**
A: Based on the provided context, I can answer your question:

**What is LangChain?**
LangChain is a